In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files.upload()

MessageError: ignored

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
omarhanyy/500-greatest-songs-of-all-time                  500 Greatest Songs of All Time                      33KB  2020-10-26 13:36:09            627  
terenceshin/covid19s-impact-on-airport-traffic            COVID-19's Impact on Airport Traffic               106KB  2020-10-19 12:40:17           2240  
sootersaalu/amazon-top-50-bestselling-books-2009-2019     Amazon Top 50 Bestselling Books 2009 - 2019         15KB  2020-10-13 09:39:21           2153  
thomaskonstantin/highly-rated-children-books-and-stories  Highly Rated Children Books And Stories            106KB  2020-10-24 12:09:59            440  
tunguz/euro-parliament-proceedings-1996-2011              Euro Parliament Proceedi

In [ ]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

100% 3.10G/3.10G [00:54<00:00, 23.1MB/s]
100% 3.10G/3.10G [00:54<00:00, 61.5MB/s]


In [ ]:
!unzip breast-histopathology-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2351_class0.png  
  inflating: IDC_regular_

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten,Conv2D,MaxPooling2D,Dropout,Flatten
from keras.models import Model,Sequential
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from os import listdir
import shutil

In [ ]:
folder=listdir('/content/IDC_regular_ps50_idx5')

In [ ]:
folder

['13402',
 '9041',
 '10279',
 '14082',
 '9173',
 '9261',
 '12821',
 '14192',
 '16553',
 '10254',
 '10301',
 '10305',
 '14213',
 '9381',
 '9345',
 '12882',
 '12909',
 '13462',
 '12949',
 '12947',
 '12951',
 '9036',
 '9228',
 '10273',
 '12242',
 '13613',
 '8913',
 '10278',
 '15512',
 '16569',
 '12901',
 '8974',
 '12626',
 '13019',
 '16532',
 '10299',
 '12875',
 '9178',
 '9044',
 '13617',
 '9181',
 '12824',
 '9291',
 '9073',
 '9325',
 '14189',
 '9321',
 '10285',
 '8914',
 '13691',
 '12878',
 '12881',
 '13401',
 '10290',
 '8867',
 '15515',
 '9346',
 '9043',
 '10261',
 '9123',
 '16568',
 '8959',
 '9265',
 '13458',
 '12748',
 '15902',
 '15632',
 '9323',
 '13106',
 '9322',
 '12751',
 '12877',
 '9256',
 '9254',
 '12929',
 '14191',
 '13694',
 '16165',
 '13459',
 '13916',
 '14157',
 '13693',
 '14304',
 '16014',
 '9347',
 '10264',
 '13020',
 '10274',
 '13687',
 '8975',
 '14153',
 '12817',
 '9225',
 '9135',
 '14188',
 '16554',
 '15513',
 '9320',
 '9258',
 '12893',
 '9324',
 '16166',
 '12955',
 '13

In [ ]:
total_images = 0
base_path='/content/IDC_regular_ps50_idx5'
for i in range(len(folder)):
  patient_id = folder[i]
  for c in [0, 1]:
    patient_path = base_path + '/'+ patient_id 
    class_path = patient_path + "/" + str(c) + "/"
    subfiles = listdir(class_path)
    total_images += len(subfiles)

In [ ]:
total_images

277524

In [ ]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + '/' + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,13402,/content/IDC_regular_ps50_idx5/13402/0/13402_i...,0
1,13402,/content/IDC_regular_ps50_idx5/13402/0/13402_i...,0
2,13402,/content/IDC_regular_ps50_idx5/13402/0/13402_i...,0
3,13402,/content/IDC_regular_ps50_idx5/13402/0/13402_i...,0
4,13402,/content/IDC_regular_ps50_idx5/13402/0/13402_i...,0


In [ ]:
print(len(data[data['target']==1]))

78786


In [ ]:
mkdir train_dataset

In [ ]:
mkdir test_dataset

In [ ]:
os.mkdir('/content/train_dataset/malicious')
os.mkdir('/content/train_dataset/benign')

In [ ]:
os.mkdir('/content/test_dataset/malicious')
os.mkdir('/content/test_dataset/benign')

In [ ]:
from PIL import Image
import os, sys

for i in np.arange(0,190000):
  im=Image.open(data.loc[i]['path'])
  rgb_im=im.convert('RGB')
  if(data.loc[i]['target']==1):
    rgb_im.save('/content/train_dataset/malicious/breast'+str(data.loc[i]['target'])+str(i)+'.jpg')
  else:
    rgb_im.save('/content/train_dataset/benign/breast'+str(data.loc[i]['target'])+str(i)+'.jpg')


In [ ]:
for i in np.arange(190000,len(data)):
  im=Image.open(data.loc[i]['path'])
  rgb_im=im.convert('RGB')
  if(data.loc[i]['target']==1):
    rgb_im.save('/content/test_dataset/malicious/breast'+str(data.loc[i]['target'])+str(i)+'.jpg')
  else:
    rgb_im.save('/content/test_dataset/benign/breast'+str(data.loc[i]['target'])+str(i)+'.jpg')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/train_dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/test_dataset',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 190000 images belonging to 2 classes.
Found 87524 images belonging to 2 classes.


In [ ]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=[224,224,3]))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(training_set,validation_data=test_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
5938/5938 [==============================] - 2222s 374ms/step - loss: 0.6468 - accuracy: 0.7210 - val_loss: 0.6508 - val_accuracy: 0.7064
Epoch 2/5
5938/5938 [==============================] - 2204s 371ms/step - loss: 0.5880 - accuracy: 0.7206 - val_loss: 0.6556 - val_accuracy: 0.7064
Epoch 3/5
5938/5938 [==============================] - 2229s 375ms/step - loss: 0.5933 - accuracy: 0.7205 - val_loss: 0.6095 - val_accuracy: 0.7064
Epoch 4/5
5938/5938 [==============================] - 2161s 364ms/step - loss: 0.5869 - accuracy: 0.7206 - val_loss: 0.6192 - val_accuracy: 0.7064
Epoch 5/5
5938/5938 [==============================] - 2152s 362ms/step - loss: 0.5856 - accuracy: 0.7206 - val_loss: 0.6485 - val_accuracy: 0.7064


In [ ]:
IMAGE_SIZE=[224,224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False
folders = glob('/content/train_dataset/*')
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model_1 = Model(inputs=vgg.input, outputs=prediction)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
model_1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
model_1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_1.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1485/1485 [==============================] - 2087s 1s/step - loss: 0.4235 - accuracy: 0.8149 - val_loss: 0.4148 - val_accuracy: 0.8212
Epoch 2/10
1485/1485 [==============================] - 2050s 1s/step - loss: 0.4073 - accuracy: 0.8230 - val_loss: 0.4115 - val_accuracy: 0.8206
Epoch 3/10
1485/1485 [==============================] - 2113s 1s/step - loss: 0.4011 - accuracy: 0.8250 - val_loss: 0.4163 - val_accuracy: 0.8200
Epoch 4/10
1485/1485 [==============================] - 2215s 1s/step - loss: 0.4043 - accuracy: 0.8251 - val_loss: 0.4525 - val_accuracy: 0.8030
Epoch 5/10
1485/1485 [==============================] - 2175s 1s/step - loss: 0.4024 - accuracy: 0.8256 - val_loss: 0.4359 - val_accuracy: 0.8125
Epoch 6/10
1485/1485 [==============================] - 2140s 1s/step - loss: 0.3952 - accuracy: 0.8284 - val_loss: 0.4806 - val_accuracy: 0.7963
Epoch 7/10
1485/1485 [==========================

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
model_3=EfficientNetB0(include_top=False,weights="imagenet",input_shape=(224, 224, 3),pooling='max',classes=1000)
folders = glob('/content/train_dataset/*')

x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

Yashvi = Model(inputs=model_3.input, outputs=prediction)
Yashvi.summary()

16711680/16705208 [==============================] - 0s 0us/step
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________

In [ ]:
Yashvi.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
Yashvi.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

In [ ]:
def preprocess_input(x):
  img=keras.applications.xception.preprocess_input(img_to_array(x))
  return array_to_img(x)

In [ ]:
IMAGE_SIZE = [299, 299]


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory('/content/train_dataset',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/test_dataset',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 190000 images belonging to 2 classes.
Found 87524 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import Xception
model_3=Xception(include_top=False,weights="imagenet",input_shape=(224, 224, 3),pooling='max',classes=1000)
folders = glob('/content/train_dataset/*')

x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

Yashvi = Model(inputs=model_3.input, outputs=prediction)
Yashvi.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________

In [ ]:
Yashvi.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']) 
Yashvi.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

In [ ]:
Yashvi.save('breastCancer.h5')